In [4]:
import pandas as pd
kaggle_input="./kaggle/input"
# kaggle_working="./kaggle/working"
kaggle_temp="/home/uc4ddc6536e59d9d8f8f5069efdb4e25/kaggle_hms/kaggle/temp1"

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)


In [5]:
import subprocess


cp_path1=f'{kaggle_temp}/f_data_cp/f_data_cp#1.csv'
cp_path=f'{kaggle_temp}/f_data_cp'
cp_size=5000
def checkpoint_data(f_data):
    try:
        display("checkpoint data=",f_data)
        cps=str(subprocess.check_output(f"ls '{cp_path}' | grep 'f_data_cp#'",shell=True,text=True))
        cps=cps.strip('\n').split('\n')
        cps.sort(key=lambda x : int(x.split('#')[1].split('.')[0]))
        print(cps)
           
        if len(cps)==1:
            cp=cps[0]
            cp_num=int(cp.split('#')[1].split('.')[0])
            new_cp_path=cp_path1.replace('#1',f'#{cp_num+1}')
            prev_cp=pd.read_csv(cp_path1.replace('#1',f'#{cp_num}'))
            pcp_index=prev_cp.index[-1]
            f_data[pcp_index:].to_csv(new_cp_path,index=False)
            
        else:
            cp=cps[-1]
            cp_num=int(cp.split('#')[1].split('.')[0])
            new_cp_path=cp_path1.replace('#1',f'#{cp_num+1}')
            prev_cp=pd.read_csv(cp_path1.replace('#1',f'#{cp_num}'))
            pcp_index=prev_cp.index[-1]
            f_data.to_csv(new_cp_path,index=False)
            # print(subprocess.check_output(f"rm '{cp_path.replace('#1',f'#{cp_num}')}'",shell=True,text=True))
    except Exception as e:
        print(e)
        print("creating 1st cp")
        new_cp_path=cp_path1
        f_data.to_csv(new_cp_path,index=False)


def load_checkpoint_data():
    try:
        cps=subprocess.check_output(f"ls {cp_path} | grep 'f_data_cp#'",shell=True,text=True)
        cps=cps.strip('\n').split('\n')
        cps.sort(key=lambda x : int(x.split('#')[1].split('.')[0]))
        f_data=pd.DataFrame()
        for cp in cps:
            cp_num=int(cp.split('#')[1].split('.')[0])
            new_cp_path=cp_path1.replace('#1',f'#{cp_num}')
            cpdf=pd.read_csv(new_cp_path)
            if len(cpdf) not in [cp_size,500,5000,50000] and len(cpdf)<50000:
                print(subprocess.check_output(f"rm '{cp_path1.replace('#1',f'#{cp_num}')}'",shell=True,text=True))
                break
            else:
                f_data=pd.concat([f_data,cpdf],axis=0,ignore_index=True)






        # for batch checkpoints
        # if len(cps)==1:
        #     new_cp_path=f"{kaggle_temp}/{cps[0]}"
        # else:
        #     new_cp_path=f"{kaggle_temp}/{cps[1]}"
        # f_data=pd.read_csv(new_cp_path)
        return f_data
    except Exception as e:
        print(e)
        return pd.DataFrame([])


In [6]:
# plen=0.001
plen=1
# format test data
train_data=pd.read_csv(f"{kaggle_input}/train.csv")
test_data=pd.read_csv(f"{kaggle_input}/test.csv")
f_names=['f_train.csv','f_test.csv']
fi=0

f_data=train_data.copy() if fi==0 else test_data.copy()
if fi==0:
    f_data=f_data.loc[:len(f_data)*plen,:]

# embedding averages for spectogram
if fi==0:
    f_data_cp=load_checkpoint_data()
    start_cp=start_cpi=len(f_data_cp)
    savg_cols=[]
    tdf_avgs=pd.DataFrame()
    if len(f_data_cp)!=0:
        savgs_index=[i+1 for i in range(len(f_data_cp.columns)) if f_data_cp.columns[i]=='other_vote']
        tdf_avgs=f_data_cp.iloc[:,savgs_index[0]:]
        savg_cols=tdf_avgs.columns
else:
    start_cp=0
    savg_cols=[]
    tdf_avgs=pd.DataFrame([])
for i in range(start_cp,len(f_data)):
    # checkpointing data
    if fi==0 and i%cp_size==0 :
        # t_df=pd.concat([f_data.iloc[start_cpi:i],tdf_avgs.iloc[max(i-start_cp-cp_size,0):].set_axis([i for i in range(start_cpi,i)])],axis=1)
        t_df=pd.concat([f_data.iloc[start_cpi:i],tdf_avgs.iloc[start_cpi:i]],axis=1)
        f_data_cp=pd.concat([f_data_cp,t_df],axis=0)
        if i!=start_cp:
            checkpoint_data(f_data_cp)
        start_cpi=i

    spect_id=f_data.loc[i,'spectrogram_id']
    # print(spect_id)
    if fi==0 and f_data.loc[i,'spectrogram_sub_id']!=0:
        rdf_avgs=tdf_avgs.loc[i-1:i-1,savg_cols].copy()
    else:
        spect_data=pd.read_parquet(f"{kaggle_input}/{'train' if fi==0 else 'test'}_spectrograms/{spect_id}.parquet")
        spect_cols=spect_data.columns
        spect_cols=[spect_cols[i] for i in range(len(spect_cols)) if i%5==0 and i!=0]
        # print(spect_cols)
        # print(len(spect_cols))
        rdf_avgs=pd.DataFrame()
        for spect_col in spect_cols:
            avgi=5
            avgs=[sum(spect_data.loc[i:i+avgi,spect_col])/avgi for i in range(0,min(len(spect_data),300),avgi)]
            # print(avgs)
            # print(len(avgs))
            df_avgs=pd.DataFrame([avgs],columns=[f'{spect_col}savg{t}' for t in range(len(avgs)) ])
            rdf_avgs=pd.concat([rdf_avgs,df_avgs],axis=1)
    # display("rdf=",rdf_avgs)
    tdf_avgs=pd.concat([tdf_avgs,rdf_avgs],axis=0,ignore_index=True)
    savg_cols=tdf_avgs.columns.tolist()
    # display("tdf=",tdf_avgs)
f_data=pd.concat([f_data,tdf_avgs],axis=1)   
        

# print(len(f_data.columns))
f_data[:3]



Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.


'checkpoint data='

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,...,RP_19.92savg55,RP_19.92savg56,RP_19.92savg57,RP_19.92savg58,RP_19.92savg59
0,1628180742,0,0.0,353733,0,...,0.132,0.166,0.128,0.136,0.212
1,1628180742,1,6.0,353733,1,...,0.132,0.166,0.128,0.136,0.212
2,1628180742,2,8.0,353733,2,...,0.132,0.166,0.128,0.136,0.212
3,1628180742,3,18.0,353733,3,...,0.132,0.166,0.128,0.136,0.212
4,1628180742,4,24.0,353733,4,...,0.132,0.166,0.128,0.136,0.212
...,...,...,...,...,...,...,...,...,...,...,...
495,3636015289,37,158.0,12849827,67,...,0.292,0.276,0.318,0.454,0.464
496,3636015289,38,162.0,12849827,68,...,0.292,0.276,0.318,0.454,0.464
497,3636015289,39,172.0,12849827,69,...,0.292,0.276,0.318,0.454,0.464
498,3636015289,40,186.0,12849827,70,...,0.292,0.276,0.318,0.454,0.464


['f_data_cp#164.csv']


In [ ]:
f_data_cp[:]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,...,RP_19.92savg55,RP_19.92savg56,RP_19.92savg57,RP_19.92savg58,RP_19.92savg59
0,1628180742,0,0.0,353733,0,...,0.132,0.166,0.128,0.136,0.212
1,1628180742,1,6.0,353733,1,...,0.132,0.166,0.128,0.136,0.212
2,1628180742,2,8.0,353733,2,...,0.132,0.166,0.128,0.136,0.212
3,1628180742,3,18.0,353733,3,...,0.132,0.166,0.128,0.136,0.212
4,1628180742,4,24.0,353733,4,...,0.132,0.166,0.128,0.136,0.212
...,...,...,...,...,...,...,...,...,...,...,...
345,1076057868,7,14.0,12741486,7,...,0.044,0.058,0.064,0.048,0.054
346,1076057868,8,16.0,12741486,8,...,0.044,0.058,0.064,0.048,0.054
347,1076057868,9,18.0,12741486,9,...,0.044,0.058,0.064,0.048,0.054
348,1076057868,10,20.0,12741486,10,...,0.044,0.058,0.064,0.048,0.054


In [ ]:
# f_data_cp[:203]
tdf_avgs[7445:7455]

,LL_1.37savg0,LL_1.37savg1,LL_1.37savg2,LL_1.37savg3,LL_1.37savg4,...,RP_19.92savg55,RP_19.92savg56,RP_19.92savg57,RP_19.92savg58,RP_19.92savg59


In [ ]:
if fi==0:
    f_data=load_checkpoint_data()
# keeping cols
savgs_index=[i+1 for i in range(len(f_data.columns)) if f_data.columns[i]=='other_vote']
savg_cols=list(f_data.columns[savgs_index[0]:])
# print(savg_cols)
vote_cols=['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']
# savg_cols=[f'savg{i}' for i in range(60)]
f_data=f_data[[col for col in vote_cols+savg_cols] if fi==0 else [col for col in ['eeg_id']+savg_cols]]
if fi==0:
    # converting votes to probs
    for i in range(len(f_data)):
        sum_votes=sum(f_data.loc[i,vote_cols])
        f_data.loc[i,vote_cols]=f_data.loc[i,vote_cols].apply(lambda x: (x/sum_votes) )

# saving to csv
f_data.to_csv(f'{kaggle_temp}/{f_names[fi]}',index=False)

In [ ]:
f_data

,eeg_id,LL_1.37savg0,LL_1.37savg1,LL_1.37savg2,LL_1.37savg3,...,RP_19.92savg55,RP_19.92savg56,RP_19.92savg57,RP_19.92savg58,RP_19.92savg59
0,3911565283,5.772,4.608,3.88,5.472,...,0.156,0.19,0.218,0.368,0.368
